In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import ast 

In [2]:
def scrap (id):
    url = 'https://brainout.org/user/' + str(id)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    #68 lvl check
    is_68 = soup.find('p', {'class': "text-yellow"})
    if is_68 == None:
        d = {'level': 0}
        df_user = pd.DataFrame(d,index=[id])
        return df_user
    #elif is_68.prettify().find('68') == -1:
    #    return None

    script_all = soup.findAll('script', type='application/javascript')

    user_stats = script_all[-1].prettify()

    start_with_user = user_stats.find('USER_STATS')
    if start_with_user == -1:
        d = {'level': 0}
        df_user = pd.DataFrame(d,index=[id])
        return df_user
    else:
        start_with = user_stats.find('{',start_with_user)
        end_with = user_stats.find('}',start_with_user)
        cut_user_stats = user_stats[start_with:end_with+1]

        dict_of_stats = ast.literal_eval(cut_user_stats)

        waste_columns = ['skills-with-weapon-','durability-of-weapon-', 'headshots-from-',
                        'longshots-from-','shots-from-weapon-','quest-','case-', 'item-case-',
                        'kills-from-bullet-', 'parts-of-', 'blueprint-','freeplay-taken-']

        future_deleted_columns =[]
        for key in dict_of_stats:
            if key.startswith(tuple(waste_columns)):
                future_deleted_columns.append(key)
        for key in future_deleted_columns:
            del dict_of_stats[key]
        df_user = pd.DataFrame(dict_of_stats, index=[id])
        return df_user

In [3]:
import numpy as np
ids = np.arange(1,1016000)

In [6]:
%%time
import multiprocessing.dummy as multiprocessing

p = multiprocessing.Pool(100)

results = p.map(scrap,ids)
p.close()
p.join()

Wall time: 4h 53min 49s


In [7]:
%%time
user_stats = list()
for i in range(100):
    k = 10160
    if i == 99:
        user_stats.append(pd.concat(results[i*k:-1],sort=False))
    else:
        user_stats.append(pd.concat(results[i*k:(i+1)*k],sort=False))

Wall time: 3h 23min 41s


In [8]:
%%time
fin_stats = pd.concat(user_stats,sort=False)
fin_stats

Wall time: 19.1 s


,ru,kpd,gears,kills,level,score,shots,deaths,rating,candies,...,kills-from-weapon-svds,kills-from-weapon-svt40,kills-from-weapon-pp2000,kills-from-weapon-mosinobrez,cheat-reports,lobby-spammer,kills-from-weapon-qbu88,kills-bullet-tag-flare,kills-from-weapon-flare,item-vietnam-1-purchased
1,58650.0,1.341507,218.0,6018.0,42.0,83645.0,104144.0,4486.0,895.0,3158.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9767102.0,1.152831,15473.0,100890.0,68.0,2460868.0,1146121.0,87515.0,26787.0,1227.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,818445.0,1.278873,13951.0,99472.0,68.0,1590408.0,1000793.0,77781.0,27600.0,5818.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,324951.0,1.103322,1061.0,24742.0,68.0,656462.0,309004.0,22425.0,5242.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015994,NaN,0.000000,16.0,281.0,10.0,3880.0,2651.0,190.0,67.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015995,NaN,0.000000,2.0,57.0,7.0,1780.0,621.0,47.0,14.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015996,8180.0,0.000000,30.0,208.0,7.0,2100.0,4535.0,224.0,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015997,6000.0,0.000000,NaN,11.0,1.0,85.0,346.0,12.0,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
fin_stats.to_csv("fin_stats_noindex.csv", index = False)